In [ ]:
X11.options(type='Xlib')

In [ ]:
library(repr)
library(iMKT)
library(dplyr)  
library(data.table)
library(ggplot2)
library(cowplot)
source('/home/jmurga/mkt/201902/scripts/src/concatenation.R')
source('/home/jmurga/mkt/201902/scripts/src/plotStyle.R')
PopFlyData <- fread('/home/jmurga/mkt/201902/results/dmelData.tab', header = T, sep = "\t")
PopHumanData <- fread('/home/jmurga/mkt/201902/results/humanData.tab', header = T, sep = "\t")

In [ ]:
PopFlyData <- fread('/home/jmurga/mkt/201902/results/dmelData.tab', header = T, sep = "\t")
PopHumanData <- fread('/home/jmurga/mkt/201902/results/humanData.tab', header = T, sep = "\t")

**Estimating new weakly deleterious fractions through differents concatenated subset of genes. *b* is defined as the area under the asymptotic curve from 0 to frequency value where alpha is equal to the lowest value on confidence intervals. *wdOnConcatenation()* returns a list of N subset containing CI values and *b* estimations after 100 replicas.**

### Ral

In [ ]:
options(repr.plot.width=5, repr.plot.height=5)

In [ ]:
outRal <- wdOnConcatenation(sizes=c(1000),pop='RAL',data=PopFlyData)
# outRal <- wdOnConcatenation(sizes=c(500,750,1000,2500,5000,7500),pop='RAL',data=PopFlyData)

# save(outRal,file='/home/jmurga/mkt/201902/results/b/ciRal.RData')
# load('/home/jmurga/mkt/201902/results/b/ciRal.RData')

In [ ]:
ciValuesRal <- data.table()

for(i in names(outRal)){
    # Save plots
    svg(paste0('/home/jmurga/mkt/201902/results/b/wdRal',i,'.svg'))
    print(outRal[[i]][['plot']])
    dev.off()
   
    # Save mean CI and sd by subset
    tmp <- data.table(mean(outRal[[i]][['ci']][,1],na.rm=T),mean(outRal[[i]][['ci']][,3]-outRal[[i]][['ci']][,2],na.rm=T),sd(outRal[[i]][['ci']][,3]-outRal[[i]][['ci']][,2],na.rm=T),as.numeric(i),'RAL')
    ciValuesRal <- rbind(ciValuesRal,tmp)
    
}

colnames(ciValuesRal) <- c('alphaMean','meanCi','sdCi','subsetLength','pop');ciValuesRal

#### One iteration

In [ ]:
# Subset genes and create list of 3500 subset
genes <- unique(PopFlyData[['id']]) %>% as.matrix()
set.seed(13753)
subsetGenes <- sample(genes,size=5000)

bootSamplesBin <- samplingGenes(geneList = subsetGenes,B=3500,bins=5000)
data=PopFlyData;sampling = bootSamplesBin;bins = 5000;population='RAL'

data <- data %>% as.data.table;data <- data[pop==population];data$id <- as.character(data$id)

# Define local variables and select iteration
output <- data.frame()
xlow <- 0; xhigh <- 1; i<-10

sampling[[i]] <- sampling[[i]] %>% as.data.frame
colnames(sampling[[i]]) <- 'id'

# SubsetData and create Daf/Div variables
subsetData <- merge(sampling[[i]],data,by='id',all.x=T)

divergence <- subsetData[,c('mi', 'di', 'm0', 'd0')]
colnames(divergence) <- c('mi', 'Di', 'm0', 'D0')
divergence <- divergence %>% summarize(mi=sum(mi),Di=sum(Di),m0=sum(m0),D0=sum(D0))

sfs0f <- as.data.frame(subsetData[['DAF0f']])
colnames(sfs0f) <- 'sfs0f'
sfs0f <- cSplit(sfs0f, 'sfs0f', ';') %>% colSums
sfs0f <- as.numeric(sfs0f)

sfs4f <- as.data.frame(subsetData[['DAF4f']]) 
colnames(sfs4f) <- 'sfs4f'
sfs4f <- cSplit(sfs4f, 'sfs4f', ';') %>% colSums
sfs4f <- as.numeric(sfs4f)

daf <- cbind(daf=seq(0.05,1,0.05), sfs0f, sfs4f) %>% as.data.frame
colnames(daf) <- c('daf','Pi','P0')

D0 <- divergence$D0
Di <- divergence$Di
Pi <- sum(daf$Pi)
P0 <- sum(daf$P0)

# Execute iMKT and plot results
resultsRal <- iMKT(daf=daf,div=divergence,xlow=xlow,xhigh=xhigh,plot=T);resultsRal

### CEU

In [ ]:
outCeu <- wdOnConcatenation(sizes=c(5000),pop='CEU',data=PopHumanData)
# outCeu <- wdOnConcatenation(sizes=c(1000,2500,5000,7500),pop='CEU',data=PopHumanData)
# save(outCeu,file='/home/jmurga/mkt/201902/results/b/ciCeu.RData')
load('/home/jmurga/mkt/201902/results/b/ciCeu.RData')

In [ ]:
ciValuesCeu <- data.table()

for(i in names(outCeu)){
    # Save plots
    svg(paste0('/home/jmurga/mkt/201902/results/b/wdCeu',i,'.svg'))
    print(outCeu[[i]]['plot'])
    dev.off()
    
    # Save mean CI and sd by subset
    tmp <- data.table(mean(outCeu[[i]][['ci']][,1],na.rm=T),mean(outCeu[[i]][['ci']][,3]-outCeu[[i]][['ci']][,2],na.rm=T),sd(outCeu[[i]][['ci']][,3]-outCeu[[i]][['ci']][,2],na.rm=T),as.numeric(i),'Ceu')
    ciValuesCeu <- rbind(ciValuesCeu,tmp)
    
}

colnames(ciValuesCeu) <- c('alphaMean','meanCi','sdCi','subsetLength','pop');ciValuesCeu

#### One iteration

In [ ]:
# Subset genes and create list of 3500 subset
genes <- unique(PopHumanData[['id']]) %>% as.matrix()
set.seed(13753)
subsetGenes <- sample(genes,size=5000)

bootSamplesBin <- samplingGenes(geneList = subsetGenes,B=3500,bins=5000)
data=PopHumanData;sampling = bootSamplesBin;bins = 5000;population='CEU'

data <- data %>% as.data.table;data <- data[pop==population];data$id <- as.character(data$id)

# Define local variables and select iteration
output <- data.frame()
xlow <- 0; xhigh <- 1; i<-10

sampling[[i]] <- sampling[[i]] %>% as.data.frame
colnames(sampling[[i]]) <- 'id'

# SubsetData and create Daf/Div variables
subsetData <- merge(sampling[[i]],data,by='id',all.x=T)

divergence <- subsetData[,c('mi', 'di', 'm0', 'd0')]
colnames(divergence) <- c('mi', 'Di', 'm0', 'D0')
divergence <- divergence %>% summarize(mi=sum(mi),Di=sum(Di),m0=sum(m0),D0=sum(D0))

sfs0f <- as.data.frame(subsetData[['DAF0f']])
colnames(sfs0f) <- 'sfs0f'
sfs0f <- cSplit(sfs0f, 'sfs0f', ';') %>% colSums
sfs0f <- as.numeric(sfs0f)

sfs4f <- as.data.frame(subsetData[['DAF4f']]) 
colnames(sfs4f) <- 'sfs4f'
sfs4f <- cSplit(sfs4f, 'sfs4f', ';') %>% colSums
sfs4f <- as.numeric(sfs4f)

daf <- cbind(daf=seq(0.05,1,0.05), sfs0f, sfs4f) %>% as.data.frame
colnames(daf) <- c('daf','Pi','P0')

D0 <- divergence$D0
Di <- divergence$Di
Pi <- sum(daf$Pi)
P0 <- sum(daf$P0)

# Execute iMKT and plot results
resultsCeu <- iMKT(daf=daf,div=divergence,xlow=xlow,xhigh=xhigh,plot=T);resultsCeu

### Merge and save CI values

In [ ]:
ciValues <- rbind(ciValuesRal,ciValuesCeu);ciValues
fwrite(ciValues,file=)